In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import MaxPooling2D
from keras_preprocessing import image
import numpy as np

In [4]:
classifier = Sequential()
classifier.add(Conv2D(64, (3,3), input_shape= (100, 100, 3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(64, (3,3), input_shape= (100, 100, 3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Flatten())

classifier.add(Dense(units=128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units=128, activation= 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units=1, activation='sigmoid'))

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
training_generator = ImageDataGenerator(rescale=1./255, rotation_range=7, horizontal_flip=True, 
                                        shear_range=0.2, height_shift_range=0.07, zoom_range=0.2)



test_generator = ImageDataGenerator(rescale=1./255)

In [10]:
training_base= training_generator.flow_from_directory('chest_xray/train',target_size=(100,100),batch_size=1, 
                                                      class_mode='binary')

Found 5216 images belonging to 2 classes.


In [11]:
test_base = test_generator.flow_from_directory('chest_xray/test', target_size=(100, 100),batch_size=1, 
                                               class_mode='binary')



Found 624 images belonging to 2 classes.


In [12]:
classifier.fit_generator(training_base,steps_per_epoch=5216/1, epochs=15, validation_data= test_base,
                         validation_steps=624/1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
5216/5216 [==============================] - 767s 147ms/step - loss: 1.2163 - accuracy: 0.7464 - val_loss: 0.5637 - val_accuracy: 0.6218
Epoch 2/15
5216/5216 [==============================] - 760s 146ms/step - loss: 0.5062 - accuracy: 0.8020 - val_loss: 1.4277 - val_accuracy: 0.6651
Epoch 3/15
5216/5216 [==============================] - 744s 143ms/step - loss: 0.3491 - accuracy: 0.8683 - val_loss: 2.3454 - val_accuracy: 0.8077
Epoch 4/15
5216/5216 [==============================] - 788s 151ms/step - loss: 0.3046 - accuracy: 0.8938 - val_loss: 0.3639 - val_accuracy: 0.8718
Epoch 5/15
5216/5216 [==============================] - 883s 169ms/step - loss: 0.2710 - accuracy: 0.9009 - val_loss: 0.4959 - val_accuracy: 0.8814
Epoch 6/15
5216/5216 [==============================] - 821s 157ms/step - loss: 0.2250 - accuracy: 0.9164 - val_loss: 0.3556 - val_accuracy: 0.8846
Epoch 7/15
5216/5216 [==============

In [20]:
model_saved = classifier.save('/home/andre/Projects/TensorFlow/model/model_classifier')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /home/andre/Projects/TensorFlow/model/model_classifier/assets


In [2]:
from keras.models import load_model

In [3]:
test = load_model('/home/andre/Projects/TensorFlow/model/model_classifier')

In [8]:
def classify_image(path_image):
    from keras_preprocessing import image
    import numpy as np
    test_image = image.load_img(path_image, target_size=(100, 100))
    test_image = image.img_to_array(test_image)
    test_image /= 255
    test_image = np.expand_dims(test_image, axis=0)   
    
    prediction = test.predict(test_image)
    prediction1 = prediction[0][0] * 100

    if prediction > 0.98:
        print('Pneumonia')
    else: 
        print('Normal X ray')

In [9]:
image = classify_image('chest_xray/val/validation/pneumonia_covid.jpeg')
image

Pneumonia


In [10]:
image = classify_image('chest_xray/val/NORMAL/NORMAL2-IM-1436-0001.jpeg')
image

Normal X ray


In [11]:
image = classify_image('chest_xray/val/PNEUMONIA/person1950_bacteria_4881.jpeg')
image

Pneumonia
